In [23]:
import pandas as pd
import numpy as np
import json
import os
import capstone_functions as cf
import capstone_json_functions as cjf
import sql_functions as sf
from pathlib import Path
import re


In [24]:
directory = "data/kickstarter_json/"
filename = "2022-06-09.json"
file_path = f"{directory}{filename}"
schema = 'kickstarter_bg'
table_name = 'boardgames'
engine = sf.get_engine()


Rename files, if they start with "Kickstarter..."

In [15]:
cjf.rename_kickstarter_files(directory)


Read the file and store it into a list called "lines"

In [16]:
# lines is a list of strings
lines = cjf.list_of_lines(file_path)


build a dictionairy by looking at the "category"

In [ ]:
list_of_categories = cjf.list_of_categories(file_path)


Build the dataframe

In [17]:
df = cjf.build_df_from_lines_list(lines)


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4129 entries, 0 to 0
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   friends                     64 non-null     object 
 1   is_starred                  64 non-null     object 
 2   is_backing                  64 non-null     object 
 3   permissions                 64 non-null     object 
 4   name                        4129 non-null   object 
 5   goal                        4129 non-null   int64  
 6   pledged                     4129 non-null   float64
 7   state                       4129 non-null   object 
 8   slug                        4129 non-null   object 
 9   country                     4129 non-null   object 
 10  country_displayable_name    4129 non-null   object 
 11  currency                    4129 non-null   object 
 12  deadline                    4129 non-null   int64  
 13  created_at                  4129 non

List of Columns to keep

In [25]:
columns_to_keep = cjf.columns_to_keep

Overwrite the df with itself with the chosen columns

In [26]:
df = df[columns_to_keep]


In [27]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4129 entries, 0 to 0
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      4129 non-null   object 
 1   goal                      4129 non-null   int64  
 2   pledged                   4129 non-null   float64
 3   state                     4129 non-null   object 
 4   slug                      4129 non-null   object 
 5   country                   4129 non-null   object 
 6   country_displayable_name  4129 non-null   object 
 7   currency                  4129 non-null   object 
 8   deadline                  4129 non-null   int64  
 9   created_at                4129 non-null   int64  
 10  launched_at               4129 non-null   int64  
 11  backers_count             4129 non-null   int64  
 12  usd_pledged               4129 non-null   object 
 13  current_currency          4129 non-null   object 
 14  creator_id 

# And now with all the json files

In [ ]:
pathlist = Path(directory).glob('**/*.json')
df_all = pd.DataFrame()
counter = 0
for path in pathlist:
    file_path = str(path)
    print(file_path)
    lines = cjf.list_of_lines(file_path)
    df_temp = cjf.build_df_from_lines_list(lines)
    df_all = pd.concat([df_all, df_temp])
    counter += 1
print(f"{counter} files are dataframed")


In [ ]:
df_all.drop_duplicates("name").info()


In [7]:
df_all = df_all[columns_to_keep]


NameError: name 'columns_to_keep' is not defined

In [15]:
sf.build_table(engine, table_name, df_all, schema)


The boardgames table was imported successfully.


In [16]:
forbidden_characters = [',', ';', '.', ':', '-', '+', '*', "#", "'", '!', '"',
                        '§', '$', '%%', '%', '&', '/', '[', ']', '{', '}', '\\', '=', '?', '`', '´']


In [18]:
df.head()


,name,goal,pledged,state,slug,country,country_displayable_name,currency,deadline,created_at,launched_at,backers_count,usd_pledged,current_currency,creator_id,creator_name,category_id,category_name,category_analytics_name,category_slug
0,Beyond Mission to Planet Hexx! Expansion for t...,5000,3537.0,live,beyond-mission-to-planet-hexx-expansion-for-th...,US,the United States,USD,1657288803,1651326234,1654696803,108,3537.0,USD,1476262695,JAMES A FITZPATRICK III,34,Tabletop Games,Tabletop Games,games/tabletop games
0,HERO'S JOURNEY FANTASY COLLECTION (STL) -1-,100,139.0,live,heros-journey-fantasy-collection-stl-1,ES,Spain,EUR,1656422746,1651092804,1654694746,12,148.47556189,USD,595822799,Luis Moreno,34,Tabletop Games,Tabletop Games,games/tabletop games
0,Dinositters! 🦕,20000,11663.0,live,dinositters,US,the United States,USD,1656434107,1648324277,1654619707,253,11663.0,USD,228767892,Joel Bodkin,34,Tabletop Games,Tabletop Games,games/tabletop games
0,Space Bug Prairie,2000,3485.0,live,space-bug-prairie,US,the United States,USD,1656431988,1652019192,1654617588,205,3485.0,USD,1589402866,Aether Studios,34,Tabletop Games,Tabletop Games,games/tabletop games
0,Till Death Do Us Part,8200,10402.0,live,till-death-do-us-part-1,US,the United States,USD,1655780400,1653059764,1654614000,335,10402.0,USD,2101297466,Douglas H. Cole,34,Tabletop Games,Tabletop Games,games/tabletop games


In [ ]:
forbidden_dict = dict()
for entry in forbidden_characters:
    forbidden_dict[entry] = ''

forbidden_dict


In [23]:
df["name"] = df["name"]replace(forbidden_dict)


TypeError: replace() missing 1 required positional argument: 'repl'

In [22]:
df.head()


,name,goal,pledged,state,slug,country,country_displayable_name,currency,deadline,created_at,launched_at,backers_count,usd_pledged,current_currency,creator_id,creator_name,category_id,category_name,category_analytics_name,category_slug
0,Beyond Mission to Planet Hexx! Expansion for t...,5000,3537.0,live,beyond-mission-to-planet-hexx-expansion-for-th...,US,the United States,USD,1657288803,1651326234,1654696803,108,3537.0,USD,1476262695,JAMES A FITZPATRICK III,34,Tabletop Games,Tabletop Games,games/tabletop games
0,HERO'S JOURNEY FANTASY COLLECTION (STL) -1-,100,139.0,live,heros-journey-fantasy-collection-stl-1,ES,Spain,EUR,1656422746,1651092804,1654694746,12,148.47556189,USD,595822799,Luis Moreno,34,Tabletop Games,Tabletop Games,games/tabletop games
0,Dinositters! 🦕,20000,11663.0,live,dinositters,US,the United States,USD,1656434107,1648324277,1654619707,253,11663.0,USD,228767892,Joel Bodkin,34,Tabletop Games,Tabletop Games,games/tabletop games
0,Space Bug Prairie,2000,3485.0,live,space-bug-prairie,US,the United States,USD,1656431988,1652019192,1654617588,205,3485.0,USD,1589402866,Aether Studios,34,Tabletop Games,Tabletop Games,games/tabletop games
0,Till Death Do Us Part,8200,10402.0,live,till-death-do-us-part-1,US,the United States,USD,1655780400,1653059764,1654614000,335,10402.0,USD,2101297466,Douglas H. Cole,34,Tabletop Games,Tabletop Games,games/tabletop games
